In [31]:
import pandas as pd
import tensorflow as tf

In [32]:
base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\CODING\DL\dl_getting_started"
rca_file = "root_cause_analysis.csv"
rca_path = f"{base_dir}\\{rca_file}"

symptom_data = pd.read_csv(rca_path)

symptom_data.head()

print(symptom_data.dtypes)
symptom_data.head()

ID                   int64
CPU_LOAD             int64
MEMORY_LEAK_LOAD     int64
DELAY                int64
ERROR_1000           int64
ERROR_1001           int64
ERROR_1002           int64
ERROR_1003           int64
ROOT_CAUSE          object
dtype: object


,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY_LEAK
1,2,0,0,0,0,0,0,1,MEMORY_LEAK
2,3,0,1,1,0,0,1,1,MEMORY_LEAK
3,4,0,1,0,1,1,0,1,MEMORY_LEAK
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


In [33]:
symptom_data['ROOT_CAUSE'].unique()

array(['MEMORY_LEAK', 'NETWORK_DELAY', 'DATABASE_ISSUE'], dtype=object)

In [34]:
symptom_data.describe()

,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,0.472000,0.418000,0.394000,0.395000,0.485000,0.432000,0.381000
std,288.819436,0.499465,0.493477,0.488879,0.489095,0.500025,0.495602,0.485876
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,500.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,750.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

label_encoder = preprocessing.LabelEncoder()
symptom_data['ROOT_CAUSE'] = label_encoder.fit_transform(
                                symptom_data['ROOT_CAUSE'])

# Convert Pandas DataFrame to a numpy vector
np_symptom = symptom_data.to_numpy().astype(float)

# Extract the feature variables (X)
X_data = np_symptom[:,1:8]

# Extract the target variable (Y) and convert to one-hot-encoding
Y_data=np_symptom[:,8]
Y_data = tf.keras.utils.to_categorical(Y_data,3)

# Split training and test data
X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, test_size=0.10)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

Shape of feature variables : (900, 7)
Shape of target variable : (900, 3)


In [36]:
from tensorflow import keras

# Training Parameters
EPOCHS=20
BATCH_SIZE=64
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

# Create a Keras sequential model
model = tf.keras.models.Sequential()

# Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(7,),
                              name='Dense-Layer-1',
                              activation='relu'))

# Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

# Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))

# Compile the model
model.compile(
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\natda\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense-Layer-1 (Dense)           │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense-Layer-2 (Dense)           │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Final (Dense)                   │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,923 (70.01 KB)

 Trainable params: 17,923 (70.01 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)


# Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset :\n---------------------------------")
model.evaluate(X_test,Y_test)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6530 - loss: 0.9655 - val_accuracy: 0.7889 - val_loss: 0.7212
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8101 - loss: 0.6452 - val_accuracy: 0.7611 - val_loss: 0.6256
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8289 - loss: 0.5325 - val_accuracy: 0.8056 - val_loss: 0.5511
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8236 - loss: 0.4697 - val_accuracy: 0.7778 - val_loss: 0.5385
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8549 - loss: 0.4268 - val_accuracy: 0.8000 - val_loss: 0.5241
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8215 - loss: 0.4492 - val_accuracy: 0.8056 - val_loss: 0.5322
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8517 - loss: 0.4017 - val_accuracy: 0.8056 - val_loss: 0.5249
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8468 - loss: 0.4563 - val_accuracy: 0.7944 - val_los

[0.43251335620880127, 0.800000011920929]

In [38]:
# Pass individual flags to predict the root cause

import numpy as np

CPU_LOAD = 1
MEMORY_LOAD = 0
DELAY = 0
ERROR_1000 = 0
ERROR_1001 = 1
ERROR_1002 = 1
ERROR_1003 = 0

# Convert input data to numpy array
input_data = np.array([[CPU_LOAD, MEMORY_LOAD, DELAY, ERROR_1000, ERROR_1001, ERROR_1002, ERROR_1003]])

# Make predictions
prediction = np.argmax(model.predict(input_data), axis=1)

# Decode prediction to get root cause
print(label_encoder.inverse_transform(prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
['DATABASE_ISSUE']


In [39]:
# Predicting as a batch

input_data = np.array([
    [1,0,0,0,1,1,0],
    [0,1,1,1,0,0,0],
    [1,1,0,1,1,0,1],
    [0,0,0,0,0,1,0],
    [1,0,1,0,1,1,1]
])

# Make batch predictions
predictions = np.argmax(model.predict(input_data), axis=1)

# Decode and print predictions
print(label_encoder.inverse_transform(predictions))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
['DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY_LEAK' 'DATABASE_ISSUE'
 'DATABASE_ISSUE']
